In [2]:
import tensorflow as tf
import os 
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import TensorBoard




# Función para preprocesar las imágenes
def preprocess(x, y):
    x = data_augmentation(x)
    x = x / 255.0
    return x, y

# Definir la ruta al directorio de datos
data_directory = '../../images'

# Configurar el aumento de datos
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
])

# Preprocesar los datos y cargarlos utilizando image_dataset_from_directory
data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    label_mode='int',
    image_size=(256, 256)
)

data = data.map(preprocess)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

train_data = data.take(train_size)
val_data = data.skip(train_size).take(val_size)
test_data = data.skip(train_size + val_size)

# Implementación de la CNN sin Transfer Learning
model = tf.keras.Sequential([
    # Capas de la CNN
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(32, activation='softmax')
])

# Compilación del modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callback para TensorBoard
log_dir = '../../logs'
tensorboard_callback = TensorBoard(log_dir=log_dir)
# Entrenamiento del modelo
history = model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[tensorboard_callback])

# Evaluación del modelo en el conjunto de prueba
evaluation = model.evaluate(test_data)
print(f'Test Accuracy (CNN sin Transfer Learning): {evaluation[1]}')

# Guardado del modelo
model.save("modelo_cnn_sin_transfer_learning.h5")

# Carga del modelo
modelo_cnn_sin_transfer_learning = tf.keras.models.load_model("modelo_cnn_sin_transfer_learning.h5")

# Función para realizar la predicción en una imagen de prueba
def predict_team(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: La imagen no se pudo cargar. Verifica la ruta y el nombre del archivo.")
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title('Imagen de Prueba')
        plt.show()
        
        resize = tf.image.resize(img, (256, 256))
        plt.imshow(resize.numpy().astype(int))
        plt.title('Imagen Redimensionada')
        plt.show()
        
        img_array = np.expand_dims(resize / 255, axis=0)
        yhat = modelo_cnn_sin_transfer_learning.predict(img_array)
        class_names = ["AC Milan", "Arsenal", "Atletico de Madrid", "Barcelona", "Bayern Munich",
                       "Benfica", "Borussia Dortmund", "Braga", "Celtic", "Copenhagen",
                       "Crvena zvezda", "Feyenoord", "Galatasaray", "Inter de Milan", "Lazio",
                       "Lens", "Manchester City", "Manchester United", "Napoli", "Newcastle United",
                       "Paris", "Porto", "PSV Eindhoven", "Rb Leipzig", "Real Madrid",
                       "Real Sociedad", "Red Bull Salzburg", "Royal Antwerp", "Sevilla",
                       "Shakhtar Donetsk", "Union Berlin", "Young Boys"]
        predicted_class_index = np.argmax(yhat[0])
        predicted_class_name = class_names[predicted_class_index]
        predicted_probability = np.max(yhat[0])
        print(f'Clase predicha (CNN sin Transfer Learning): {predicted_class_name} con una probabilidad de {predicted_probability:.2f}')

# Ruta de la imagen de prueba
image_path = '../../real_madrid_prueba.png'
predict_team(image_path)


Found 3408 files belonging to 32 classes.
Epoch 1/10


74/74 [==============================] - 72s 952ms/step - loss: 3.3145 - accuracy: 0.0756 - val_loss: 3.1604 - val_accuracy: 0.0982
Epoch 2/10
74/74 [==============================] - 81s 1s/step - loss: 2.9566 - accuracy: 0.1634 - val_loss: 2.8905 - val_accuracy: 0.2024
Epoch 3/10
74/74 [==============================] - 79s 1s/step - loss: 2.7581 - accuracy: 0.2487 - val_loss: 2.7452 - val_accuracy: 0.2560
Epoch 4/10
74/74 [==============================] - 80s 1s/step - loss: 2.5017 - accuracy: 0.3112 - val_loss: 2.5159 - val_accuracy: 0.3304
Epoch 5/10
74/74 [==============================] - 82s 1s/step - loss: 2.3052 - accuracy: 0.3640 - val_loss: 2.4001 - val_accuracy: 0.3690
Epoch 6/10
74/74 [==============================] - 163s 2s/step - loss: 2.0751 - accuracy: 0.4400 - val_loss: 2.2380 - val_accuracy: 0.4122
Epoch 7/10
74/74 [==============================] - 161s 2s/step - loss: 1.9443 - accuracy: 0.4569 - val_loss: 2

c:\Users\albab\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NameError: name 'cv2' is not defined